Search!

In [1]:
# typing — Support for gradual typing as defined by PEP 484 and subsequent PEPs.
from typing import TypeAlias, Literal, List, Tuple, Callable, Optional

# NumPy — Funções para computação numérica.
import numpy as np

# matplotlib — An object-oriented plotting library.
import matplotlib.pyplot as plt

# itertools — Functional tools for creating and using iterators.
from itertools import cycle

In [ ]:
Function: TypeAlias = Callable[[float, float], float]
"""Tipo reservado para a representação de uma função ℝ² -> ℝ."""

BoundType: TypeAlias = Tuple[float, float]
"""Tipo reservado para a representação de limites de uma variável."""

In [ ]:
color_cycle = cycle("bgrcmk")
"""Ciclo de cores para seleção "aleatória"."""

In [4]:
class Search:

	bounds: Tuple[BoundType, BoundType]
	"""Limites inferior e superior do problema (para cada variável)."""

	objective: Function
	"""Função objetivo a ser minimizada."""

	problem: str
	"""O problema é de maximização ou minimização?"""

	operator: Callable[[float, float], bool]
	"""Este é o operador do problema."""

	argf: Callable[..., int]
	"""Esta é a função de seleção do argumento mínimo/máximo."""

	def __init__(self, f: Function, x: BoundType, y: BoundType, problem: str = "min") -> None:

		# Esta é a função objetivo a ser minimizada.
		self.objective = f
		
		# Estes são os limites de cada variável.
		self.bounds = (x, y)

		# Este é o tipo de problema, por padrão, de minimização.
		self.problem = problem

		# Este é o operador de minimização.
		self.operator = lambda x, y: x < y

		# Use o `argmin` por padrão.
		self.argf = np.argmin

		# Caso o problema seja de maximização, troque o operador.
		if problem == "max":
			
			self.operator = lambda x, y: x > y
			self.argf = np.argmax

		# Pré-compute os limites inferior e superior como uma variável.
		self.lower = np.array([self.bounds[0][0], self.bounds[1][0]])
		self.upper = np.array([self.bounds[0][1], self.bounds[1][1]])

	def random_uniform_neighbor(self, vec: np.ndarray, ε: float) -> np.ndarray:
		"""
		Esta função retorna um vizinho aleatório com uma perturbação uniforme `ε`.
		"""

		# Perturbe todos os valores uniformemente em [-ε, ε).
		vec = np.add(vec, np.random.uniform(-ε, ε, size = vec.size))

		# Impeça que valores fora do espaço de estados apareçam.
		return np.clip(vec, self.lower, self.upper)

	def random_normal_neighbor(self, vec: np.ndarray, σ: float) -> np.ndarray:
		"""
		Esta função retorna um vizinho aleatório com uma perturbação normal `σ`.
		"""

		# Perturbe todos os valores normal ~N(0, σ).
		vec = np.add(vec, np.random.normal(loc = 0, scale = σ, size = vec.size))

		# Impeça que valores fora do espaço de estados apareçam.
		return np.clip(vec, self.lower, self.upper)

	def random_global_neighbor(self) -> np.ndarray:
		"""
		Esta função retorna um "vizinho" (valor qualquer) global dentro dos limites `Δ`.
		"""

		# Perturbe todos os valores uniformemente em [-ε, ε).
		return np.random.uniform(low = self.lower, high = self.upper, size = len(self.bounds))

	def hill_climbing(self, ε: float, n_neighbors: int = 10, max_iterations: int = 1_000, early_stop: int = 10, rounds: int = 100):
		"""
		Esta função realiza a subida de encosta (hill climbing) para a função.
		"""

		# Estes são todos os pontos mínimos e valores f(x, y).
		points = []
		values = []

		# Rode este algoritmo por `rounds` rodadas.
		for _ in range(rounds):

			# Inicie o algoritmo no limite inferior de todas as variáveis.
			x_optimal = np.copy(self.lower)

			# Este é o melhor score até o momento.
			best_score = self.objective(*x_optimal)
				
			# Esta variável guarda quantas iterações fazem que "não há melhoria".
			no_improvement = 0
			
			for _ in range(max_iterations):

				# Caso não haja melhora em `early_stop` iterações, pare.
				if no_improvement >= early_stop:
					break

				# Produza todos os vizinhos.
				neighbors = np.array([self.random_uniform_neighbor(x_optimal, ε) for _ in range(n_neighbors)])
				neighbor_scores = np.array([self.objective(*neighbor) for neighbor in neighbors])

				# Encontre o melhor vizinho.
				best_neighbor_idx = self.argf(neighbor_scores)
				best_neighbor_score = neighbor_scores[best_neighbor_idx]
				best_neighbor = neighbors[best_neighbor_idx]

				# Se o melhor vizinho é melhor..
				if self.operator(best_neighbor_score, best_score):

					# Atualizamos a melhor solução e houve melhora.
					x_optimal, best_score = best_neighbor, best_neighbor_score
					no_improvement = 0

				else:
					
					# Não houve melhora.
					no_improvement += 1

			# Adicione o ponto no histórico.
			points.append(x_optimal)
			values.append(best_score)

		# Retorne os resultados obtidos.
		return np.array(points), np.array(values)

	def local_random_search(self, σ: float, max_iterations: int = 1_000, early_stop: int = 10, rounds: int = 100):
		"""
		Esta função realiza a busca aleatória local (local random search) para a função.
		"""

		# Estes são todos os pontos mínimos.
		points: List[np.ndarray] = []

		# Estes são todos os valores de f(x, y) de cada ponto.
		values: List[float] = []

		# Rode este algoritmo por `rounds` rodadas.
		for _ in range(rounds):

			# Inicie o algoritmo em um ponto aleatório.
			x_optimal = self.random_global_neighbor()

			# Este é o melhor score até o momento.
			best_score: float = self.objective(*x_optimal)
				
			# Esta variável armazena a quantidade de iterações já feitas.
			iteration: int = 0

			# Esta variável guarda quantas iterações fazem que "não há melhoria".
			no_improvement: int = 0

			# Enquanto o número máximo de iterações foi alancaçado, ou houver melhoria..
			while (iteration < max_iterations) or (no_improvement == 0):

				# Caso não haja melhora em `early_stop` iterações, pare.
				if no_improvement >= early_stop:
					break

				# Gere um vizinho aleatório com perturbação normal σ.
				# Perturba uniformente, mas a distribuição é normal com desvio padrão σ.
				neighbor = self.random_normal_neighbor(σ = σ, vec = x_optimal)
				
				# Avalie o vizinho gerado aleatoriamente.
				neighbor_score = self.objective(*neighbor)

				# Se o melhor vizinho é o melhor..
				if self.operator(neighbor_score, best_score):
				
					# Atualizamos a melhor solução e houve melhora.
					x_optimal, best_score = neighbor, neighbor_score
					no_improvement = 0
				
				else:
					
					# Não houve melhora.
					no_improvement += 1
			
			# Adicione o ponto no histórico.
			points.append(x_optimal)
			values.append(best_score)

		# Retorne os resultados obtidos.
		return np.array(points), np.array(values)
	
	def global_random_search(self, max_iterations: int = 1_000, rounds: int = 100):
		"""
		Esta função realiza a busca aleatória global (global random search) para a função.
		"""

		# Estes são todos os pontos mínimos e valores f(x, y).
		points = []
		values = []

		# Rode este algoritmo por `rounds` rodadas.
		for _ in range(rounds):
			
			# Inicie o algoritmo em um ponto aleatório.
			x_optimal = self.random_global_neighbor()
			
			# Este é o melhor score até o momento.
			best_score = self.objective(*x_optimal)
				
			for _ in range(max_iterations):
				
				# Gere um "vizinho" completamente aleatório.
				neighbor = self.random_global_neighbor()
				
				# Avalie o vizinho gerado aleatoriamente.
				neighbor_score = self.objective(*neighbor)

				# Se o vizinho é melhor (sempre minimização)..
				if self.operator(neighbor_score, best_score):

					# Atualizamos a melhor solução e houve melhora.
					x_optimal, best_score = neighbor, neighbor_score

			# Adicione o ponto no histórico.
			points.append(x_optimal)
			values.append(best_score)

		# Retorne os resultados obtidos.
		return np.array(points), np.array(values)

In [9]:
class Plot:

	@staticmethod
	def histogram(data: np.ndarray, title: str | None = None, filename: str | None = None) -> None:
		"""
		Faz o plot de um histograma com os dados fornecidos.
		"""
		
		fig, ax = plt.subplots(figsize = (8, 6))

		plt.hist(data, edgecolor = 'white', color = "#6280B8")
		plt.axvline(float(np.mean(data)), color = "k", linestyle = "dashed", linewidth = 2)

		if title is not None:
			ax.set_title(title)

		plt.gcf().set_dpi(300)
		
		plt.xlabel("Valor")
		plt.ylabel("Frequência")

		plt.grid(False)

		plt.tight_layout()

		if filename is not None:
			plt.savefig("./Document/P1/" + filename + ".jpeg", dpi = 300)
		
		plt.show()

	@staticmethod
	def surface(p: Search, label: Optional[str] = None, points: Optional[np.ndarray] = None, α: float = 0.5, granularity: int = 1000, cmap: str = "jet", filename: str | None = None) -> None:
		"""
		Faz o plot de uma superfície com a função fornecida.
		"""
		
		fig = plt.figure(figsize = (8, 6))
		ax = fig.add_subplot(projection = "3d")

		# Cria uma grade de valores x e y
		x = np.linspace(*p.bounds[0], granularity)
		y = np.linspace(*p.bounds[1], granularity)
		x, y = np.meshgrid(x, y)

		# Calcula os valores z como f(*p).
		z = p.objective(x, y) # type: ignore

		# Defina um valor alto de DPI.
		plt.gcf().set_dpi(300)

		# Este será o offset de cada eixo.
		min_z = np.min(z)

		# Faça o plot da superfície.
		surf = ax.plot_surface(x, y, z, cmap = cmap, edgecolor = "none", alpha = α) # type: ignore

		# Desenhe o contorno abaixo da superfície.
		ax.contour(x, y, z, zdir = "z", offset = min_z, cmap = "Greys_r")

		# Adicione uma barra de cores.
		# fig.colorbar(surf, ax = ax, shrink = 0.5, aspect = 5)

		# Define um título, se fornecido
		if label:
			ax.set_title(label)

		if points is not None:
			ax.scatter(points[:, 0], points[:, 1], [p.objective(*point) for point in points], marker = "v", color = "k")

		plt.xlabel('$x$')
		plt.ylabel('$y$')

		plt.tight_layout()

		if filename is not None:
			plt.savefig("./Document/P1/" + filename + ".jpeg", dpi = 300)

		plt.show()

### Problema 1

In [ ]:
p = Search(
	
	# Função objetivo e restrições do espaço de estados.
	f = lambda x, y: x ** 2 + y ** 2,
	x = (-10, 10), y = (-10, 10),
    problem = "min"

)

expr = "x^2 + y^2"

Plot.surface(p, filename = "P1-Surface")

points, values = p.hill_climbing(ε = 1, n_neighbors = 10)
Plot.surface(p, points = points, filename = "P1-HC-Surface")
Plot.histogram(values, filename = "P1-HC-Histogram")

points, values = p.local_random_search(σ = 0.1)
Plot.surface(p, points = points, filename = "P1-LRS-Surface")
Plot.histogram(values, filename = "P1-LRS-Histogram")

points, values = p.global_random_search()
Plot.surface(p, points = points, filename = "P1-GRS-Surface")
Plot.histogram(values, filename = "P1-GRS-Histogram")

### Problema 2

In [ ]:
p = Search(
	
	# Função objetivo e restrições do espaço de estados.
	f = lambda x, y: np.exp(-(x**2 + y**2)) + 2 * np.exp(-((x - 1.7)**2 + (y - 1.7)**2)),
	x = (-2, 4), y = (-2, 5),
    problem = "max"

)

expr = r"e^{-((x^2 + y^2))} + 2 \cdot e^{-(x - 1.7)^2 + (y - 1.7)^2}"

Plot.surface(p, filename = "P2-Surface")

points, values = p.hill_climbing(ε = 1, n_neighbors = 10)
Plot.surface(p, points = points, filename = "P2-HC-Surface")
Plot.histogram(values, filename = "P2-HC-Histogram")

points, values = p.local_random_search(σ = 0.1)
Plot.surface(p, points = points, filename = "P2-LRS-Surface")
Plot.histogram(values, filename = "P2-LRS-Histogram")

points, values = p.global_random_search()
Plot.surface(p, points = points, filename = "P2-GRS-Surface")
Plot.histogram(values, filename = "P2-GRS-Histogram")

### Problema 3

In [ ]:
p = Search(
	
	# Função objetivo e restrições do espaço de estados.
	f = lambda x, y: -20 * np.exp(-0.2 * np.sqrt(0.5 * (x**2 + y**2))) - np.exp(0.5 * (np.cos(2 * np.pi * x) + np.cos(2 * np.pi * y))) + 20 + np.e,
	x = (-8, 8), y = (-8, 8),
    problem = "min"

)

expr = r"-20 \cdot e^{-0.2 \cdot \sqrt{0.5 \cdot (x^2 + y^2)}} - e^{0.5 \cdot (\cos(2 \pi x) + \cos(2 \pi y))} + 20 + e^1"

Plot.surface(p, filename = "P3-Surface")

points, values = p.hill_climbing(ε = 1, n_neighbors = 10)
Plot.surface(p, points = points, filename = "P3-HC-Surface")
Plot.histogram(values, filename = "P3-HC-Histogram")

points, values = p.local_random_search(σ = 0.1)
Plot.surface(p, points = points, filename = "P3-LRS-Surface")
Plot.histogram(values, filename = "P3-LRS-Histogram")

points, values = p.global_random_search()
Plot.surface(p, points = points, filename = "P3-GRS-Surface")
Plot.histogram(values, filename = "P3-GRS-Histogram")

### Problema 4

In [ ]:
p = Search(
	
	# Função objetivo e restrições do espaço de estados.
	f = lambda x, y: (x ** 2 - 10 * np.cos(2 * np.pi * x) + 10) + (y**2 - 10 * np.cos(2 * np.pi * y) + 10),
	x = (-5.12, 5.12), y = (-5.12, 5.12),
    problem = "min"

)

expr = r"( x^2 - 10 \cdot \cos( 2 \pi x ) + 10 ) + ( y^2 - 10 \cdot \cos( 2 \pi y ) + 10 )"

Plot.surface(p, filename = "P4-Surface")

points, values = p.hill_climbing(ε = 1, n_neighbors = 10)
Plot.surface(p, points = points, filename = "P4-HC-Surface")
Plot.histogram(values, filename = "P4-HC-Histogram")

points, values = p.local_random_search(σ = 0.1)
Plot.surface(p, points = points, filename = "P4-LRS-Surface")
Plot.histogram(values, filename = "P4-LRS-Histogram")

points, values = p.global_random_search()
Plot.surface(p, points = points, filename = "P4-GRS-Surface")
Plot.histogram(values, filename = "P4-GRS-Histogram")

### Problema 5

In [ ]:
p = Search(
	
	# Função objetivo e restrições do espaço de estados.
	f = lambda x, y: ((x * np.cos(x)) / 20 + 2 * np.exp(-(x**2) - ((y - 1)**2)) + 0.01 * x * y),
	x = (-10, 10), y = (-10, 10),
    problem = "min"

)

expr = r"\frac{x \cdot \cos(x)}{20} + 2 \cdot e^{-(x)^2 - (y - 1)^2} + 0.01 \cdot x_1 \cdot y"

Plot.surface(p, filename = "P5-Surface")

points, values = p.hill_climbing(ε = 1, n_neighbors = 10)
Plot.surface(p, points = points, filename = "P5-HC-Surface")
Plot.histogram(values, filename = "P5-HC-Histogram")

points, values = p.local_random_search(σ = 0.1)
Plot.surface(p, points = points, filename = "P5-LRS-Surface")
Plot.histogram(values, filename = "P5-LRS-Histogram")

points, values = p.global_random_search()
Plot.surface(p, points = points, filename = "P5-GRS-Surface")
Plot.histogram(values, filename = "P5-GRS-Histogram")

### Problema 6

In [ ]:
p = Search(
	
	# Função objetivo e restrições do espaço de estados.
	f = lambda x, y: x * np.sin(4 * np.pi * x) - y * np.sin(4 * np.pi * y + np.pi) + 1,
	x = (-1, 3), y = (-1, 3),
    problem = "max"

)

expr = r"x \cdot \sin(4 \pi x) - y \cdot \sin(4 \pi y + \pi) + 1"

Plot.surface(p, filename = "P6-Surface")

points, values = p.hill_climbing(ε = 1, n_neighbors = 10)
Plot.surface(p, points = points, filename = "P6-HC-Surface")
Plot.histogram(values, filename = "P6-HC-Histogram")

points, values = p.local_random_search(σ = 0.1)
Plot.surface(p, points = points, filename = "P6-LRS-Surface")
Plot.histogram(values, filename = "P6-LRS-Histogram")

points, values = p.global_random_search()
Plot.surface(p, points = points, filename = "P6-GRS-Surface")
Plot.histogram(values, filename = "P6-GRS-Histogram")

### Problema 7

In [ ]:
p = Search(
	
	# Função objetivo e restrições do espaço de estados.
	f = lambda x, y: -np.sin(x) * np.sin((x ** 2 / np.pi)) ** (2 * 10) - np.sin(y) * np.sin((2 * y ** 2 / np.pi)) ** (2 * 10),
	x = (0, np.pi), y = (0, np.pi),
    problem = "min"

)

expr = r"- \sin(x) \cdot \sin((\frac{x^2}{\pi}))^{2 \cdot 10} - \sin(y) \cdot \sin((\frac{2 y^2}{\pi}))^{2 \cdot 10}"

Plot.surface(p, filename = "P7-Surface")

points, values = p.hill_climbing(ε = 1, n_neighbors = 10)
Plot.surface(p, points = points, filename = "P7-HC-Surface")
Plot.histogram(values, filename = "P7-HC-Histogram")

points, values = p.local_random_search(σ = 0.1)
Plot.surface(p, points = points, filename = "P7-LRS-Surface")
Plot.histogram(values, filename = "P7-LRS-Histogram")

points, values = p.global_random_search()
Plot.surface(p, points = points, filename = "P7-GRS-Surface")
Plot.histogram(values, filename = "P7-GRS-Histogram")

### Problema 8

In [ ]:
p = Search(
	
	# Função objetivo e restrições do espaço de estados.
	f = lambda x, y: -(y + 47) * np.sin(np.sqrt(np.abs(x / 2 + (y + 47)))) - x * np.sin(np.sqrt(np.abs(x - (y + 47)))),
	x = (-200, 20), y = (-200, 20),
    problem = "min"

)

expr = r"-(y + 47) \cdot \sin(\sqrt{|x / 2 + (y + 47)|}) - x \cdot \sin(\sqrt{|x - (y + 47)|})"

Plot.surface(p, filename = "P8-Surface")

points, values = p.hill_climbing(ε = 1, n_neighbors = 10)
Plot.surface(p, points = points, filename = "P8-HC-Surface")
Plot.histogram(values, filename = "P8-HC-Histogram")

points, values = p.local_random_search(σ = 0.1)
Plot.surface(p, points = points, filename = "P8-LRS-Surface")
Plot.histogram(values, filename = "P8-LRS-Histogram")

points, values = p.global_random_search()
Plot.surface(p, points = points, filename = "P8-GRS-Surface")
Plot.histogram(values, filename = "P8-GRS-Histogram")